# TP Intelligence Artificielle - Recherche Arborescente Informée

# **Partie 0 : Visualisation d'états, Classe Taquin et Classe Node**

Nous récupérons les classes du TP précédent ainsi que la fonction visualise_state.

In [23]:
from IPython.display import display, HTML

def visualize_state(state):
    """Visualizes the given state of the Taquin using HTML."""
    html = "<table>"
    for row in state:
        html += "<tr>"
        for tile in row:
            if tile == 0:
                html += "<td style='background-color: lightgray; width: 30px; height: 30px; text-align: center; font-size: 20px;'> </td>"  # Blank tile
            else:
                html += f"<td style='background-color: lightblue; width: 30px; height: 30px; text-align: center; font-size: 20px;'>{tile}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

class Taquin:
    """
    A class representing the Taquin problem.
    """

    def __init__(self, initial_state, goal_state, size):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = size

    def actions(self, state):
        """Returns the possible actions (moves) from the given state."""
        # Find the position of the blank tile (0)
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Define possible moves (up, down, left, right)
        possible_actions = []
        if row > 0:
            possible_actions.append("up")
        if row < self.size-1:
            possible_actions.append("down")
        if col > 0:
            possible_actions.append("left")
        if col < self.size-1:
            possible_actions.append("right")

        return possible_actions

    def result(self, state, action):
        """Returns the state that results from applying the given action."""
        # Create a copy of the state to avoid modifying the original
        new_state = [list(row) for row in state]

        # Find the position of the blank tile (0)
        row, col = next(
            (r, c)
            for r, row in enumerate(state)
            for c, val in enumerate(row)
            if val == 0
        )

        # Apply the action to move the blank tile
        if action == "up":
            new_state[row][col], new_state[row - 1][col] = (
                new_state[row - 1][col],
                new_state[row][col],
            )
        elif action == "down":
            new_state[row][col], new_state[row + 1][col] = (
                new_state[row + 1][col],
                new_state[row][col],
            )
        elif action == "left":
            new_state[row][col], new_state[row][col - 1] = (
                new_state[row][col - 1],
                new_state[row][col],
            )
        elif action == "right":
            new_state[row][col], new_state[row][col + 1] = (
                new_state[row][col + 1],
                new_state[row][col],
            )

        return new_state

    def is_goal(self, state):
        return state == self.goal_state  # Directly compare with goal_state

    def cost(self, state, action):
        return 1  # Default cost is 1

class Node:
    """
    A node in a search tree.
    __init__: Initializes a node with its state, parent, action, path cost, and depth.
    __repr__: Provides a string representation of the node.
    __lt__: Defines a comparison operator for nodes based on their states.
    expand: Generates child nodes by applying all possible actions.
    child_node: Creates a single child node for a given action.
    solution: Returns the sequence of actions that led to this node.
    path: Returns the path from the root to this node as a list of nodes.
    """

    def __init__(self, state, parent=None, action=None, path_cost = 0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0 if parent is None else parent.depth + 1

    # def __repr__(self):
    #     return "<Node {}>".format(self.state)

    # def __lt__(self, other):
    #     return self.state < other.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [
            self.child_node(problem, action)
            for action in problem.actions(self.state)
        ]

    def child_node(self, problem, action):
        """Create a child node by applying the given action."""
        next_state = problem.result(self.state, action)
        next_node = Node(
            next_state,
            parent=self,
            action=action,
            path_cost = self.path_cost + problem.cost(self.state, action),
        )
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

# **Partie 1 : Heuristiques pour le jeu de Taquin**

## **1.1 H$_1$ : Nombre de tuiles mal placées**

### **Exercice 1**

Écrivez une fonction qui calcule le nombre de tuiles mal placées dans un état du jeu de Taquin

In [24]:
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]
def calcule_h1(state, goal_state):
    count = 0
    for i in range(len(goal_state)):
        for j in range(len(goal_state[i])):
            if state[i][j] != 0 and state[i][j] != goal_state[i][j]:
                count += 1
    return count
    

### **Exercice 2**

Calculez le nombre de tuiles mal placées pour les états suivants :
1. [[1, 2, 3], [4, 5, 0], [6, 7, 8]]
2. [[1, 2, 3], [0, 5, 6], [4, 7, 8]]    
3. [[1, 0, 3], [4, 2, 5], [7, 8, 6]]
4. [[1, 0, 3], [4, 5, 2], [7, 6, 8]]
5. [[1, 0, 3], [4, 2, 5], [6, 7, 8]]
6. [[1, 2, 3], [4, 0, 6], [7, 5, 8]]
7. [[1, 2, 3], [0, 4, 6], [7, 5, 8]]
8. [[1, 2, 3], [4, 6, 0], [7, 5, 8]]
9. [[1, 3, 6], [4, 2, 5], [7, 0, 8]]  
10. [[1, 3, 6], [4, 2, 0], [7, 5, 8]]  
11. [[1, 3, 6], [4, 0, 2], [7, 5, 8]]  
12. [[3, 1, 2], [4, 6, 5], [7, 0, 8]]  
13. [[8, 1, 2], [0, 4, 3], [7, 6, 5]]
14. [[1, 4, 2], [7, 0, 6], [5, 3, 8]]
15. [[2, 8, 3], [1, 6, 4], [7, 0, 5]]

In [25]:

problem_states = [
    [[1, 2, 3], [4, 5, 0], [6, 7, 8]],  # 1
    [[1, 2, 3], [0, 5, 6], [4, 7, 8]],  # 2
    [[1, 0, 3], [4, 2, 5], [7, 8, 6]],  # 3
    [[1, 0, 3], [4, 5, 2], [7, 6, 8]],  # 4
    [[1, 0, 3], [4, 2, 5], [6, 7, 8]],  # 5
    [[1, 2, 3], [4, 0, 6], [7, 5, 8]],  # 6
    [[1, 2, 3], [0, 4, 6], [7, 5, 8]],  # 7
    [[1, 2, 3], [4, 6, 0], [7, 5, 8]],  # 8
    [[1, 3, 6], [4, 2, 5], [7, 0, 8]],  # 9
    [[1, 3, 6], [4, 2, 0], [7, 5, 8]],  # 10
    [[1, 3, 6], [4, 0, 2], [7, 5, 8]],  # 11
    [[3, 1, 2], [4, 6, 5], [7, 0, 8]],  # 12
    [[8, 1, 2], [0, 4, 3], [7, 6, 5]],  # 13
    [[1, 4, 2], [7, 0, 6], [5, 3, 8]],  # 14
    [[2, 8, 3], [1, 6, 4], [7, 0, 5]]   # 15
]

def ex2_Mal_placees():
    for state in problem_states:
        print(calcule_h1(state, goal_state))


ex2_Mal_placees()

3
3
3
3
5
2
3
3
5
5
5
6
7
6
6


## **1.2 H$_2$ : Distance de Manhattan**

### **Exercice 3**
Écrivez une fonction qui calcule la distance de Manhattan entre un état et l'état but. La fonction **next()** pourrait être utile pour trouver la position d'une tuile donnée dans l'état but.

In [26]:
def calcule_h2(problem, goal_state):
    state = problem
    dist = 0
    for r, row in enumerate(state): #含索引
        for c, tile in enumerate(row):
            if tile == 0:
                continue
            gr, gc = next(
                (i, j)
                for i, grow in enumerate(goal_state)
                for j, val in enumerate(grow)
                if val == tile
            )
            dist += abs(r - gr) + abs(c - gc)
    return dist

### **Exercice 4**
1. Calculez la distance de Manhattan des états de l'exercice 2.
2. Que pouvez-vous observer par rapport aux deux heuristiques ?

In [27]:
def ex4_manhattan():
     for k,state in enumerate(problem_states):
        h1 = calcule_h1(state, goal_state)
        h2 = calcule_h2(state, goal_state)
        print(f"num = {k}: h1 = {h1}, h2 = {h2}")
ex4_manhattan()

num = 0: h1 = 3, h2 = 5
num = 1: h1 = 3, h2 = 3
num = 2: h1 = 3, h2 = 3
num = 3: h1 = 3, h2 = 5
num = 4: h1 = 5, h2 = 7
num = 5: h1 = 2, h2 = 2
num = 6: h1 = 3, h2 = 3
num = 7: h1 = 3, h2 = 3
num = 8: h1 = 5, h2 = 5
num = 9: h1 = 5, h2 = 5
num = 10: h1 = 5, h2 = 6
num = 11: h1 = 6, h2 = 7
num = 12: h1 = 7, h2 = 11
num = 13: h1 = 6, h2 = 10
num = 14: h1 = 6, h2 = 9


# **Partie 2 : Algorithme A étoile**

### **Exercice 5**

Créer une fonction A_etoile qui prend un objet problem en entrée ainsi qu'une heuristique et exécute l'algorithme A.*

Votre fonction doit :

1. Retourner le nœud objectif trouvé ainsi que le nombre de nœuds explorés.
2. Retourner None si l'algorithme explore tout l'arbre sans trouver le nœud objectif.
3. Prendre en compte un budget d'exploration pour arrêter l'algorithme si aucune solution n'est trouvée après plusieurs itérations (utile pour les grands problèmes). Le budget sera également un paramètre d'entrée de la fonction, avec float('inf') comme valeur par défaut.

Pour la **frontière** et l'ensemble des **nœuds déjà explorés**, vous pouvez utiliser des listes. Cependant, la frontière doit contenir les nœuds à explorer ainsi que leurs coûts (coût réel + heuristique). Pour trouver l'élément (nœud, coût) de coût minimal dans la frontière, effectuez une boucle **for** pour trouver l'indice, puis utilisez pop.

Pour ajouter un élément à une liste, vous pouvez utiliser **liste.append(element)**.

Rappelez-vous que la vérification **is_goal** doit être effectuée avant d'explorer les enfants.

In [28]:

def A_etoile(problem, heuristique, budget=float('inf')):

    node = Node(problem.initial_state)
    # f(n) = g(n) + h(n)
    # g(n) = 0 (node.path_cost)
    h_val = heuristique(node.state, problem.goal_state)
    f_val = node.path_cost + h_val
    
    # Frontier (node, f_val)
    frontier = [(node, f_val)]
    explored = []  
    count = 0     

    while frontier and count < budget:
        
        # Frontier -> f min
        min_idx = 0
        for i in range(1, len(frontier)):
            if frontier[i][1] < frontier[min_idx][1]:
                min_idx = i
        
        # pop min
        node, f_val = frontier.pop(min_idx)
        count += 1
        
        # verify is_goal before exploring children
        if problem.is_goal(node.state):
            return node, count
        
        explored.append(node.state)
        
        # -child
        for child in node.expand(problem):
            # check child
            if child.state in explored:
                continue
            
            # f = g + h
            # g: child.path_cost already expand -> child_node : (parent.cost + step_cost)
            h_child = heuristique(child.state, problem.goal_state)
            f_child = child.path_cost + h_child
            
            # if child in frontier
            in_frontier = False
            for i in range(len(frontier)):
                existing_node, existing_f = frontier[i]
                if existing_node.state == child.state:
                    in_frontier = True
                    # available alternative: if new path is better, update frontier
                    if f_child < existing_f:
                        frontier[i] = (child, f_child)
                    break
            
            # if not in frontier, add
            if not in_frontier:
                frontier.append((child, f_child))
                
    return None, count

"""
不考虑真实的步数 g(n)，只看启发式 h(n) 的 贪婪最佳优先搜索 (Greedy Best-First Search)。
h2 - > h1
在 Python 里可以利用元组 (h2, h1) 的比较特性：

def Greedy_Custom(problem, budget=float('inf')):
    node = Node(problem.initial_state)
    # 不用传 heuristique 参数，直接算两个
    h1 = calcule_h1(node.state, problem.goal_state)
    h2 = calcule_h2(node.state, problem.goal_state)
    
    # Frontier 存放 (节点, (h2, h1))
    frontier = [(node, (h2, h1))]
    # ... 省略中间代码 ...
    
    # 找最小值的逻辑：
    min_idx = 0
    for i in range(1, len(frontier)):
        # Python 比较 (h2, h1) 时，会自动先比较 h2，如果 h2 相等才会去比较 h1
        if frontier[i][1] < frontier[min_idx][1]: 
            min_idx = i
            
    # ... 扩展子节点时加入 Frontier 的逻辑：
    frontier.append((child, (child_h2, child_h1)))

===================================================================
"""

"\n不考虑真实的步数 g(n)，只看启发式 h(n) 的 贪婪最佳优先搜索 (Greedy Best-First Search)。\nh2 - > h1\n在 Python 里可以利用元组 (h2, h1) 的比较特性：\n\ndef Greedy_Custom(problem, budget=float('inf')):\n    node = Node(problem.initial_state)\n    # 不用传 heuristique 参数，直接算两个\n    h1 = calcule_h1(node.state, problem.goal_state)\n    h2 = calcule_h2(node.state, problem.goal_state)\n\n    # Frontier 存放 (节点, (h2, h1))\n    frontier = [(node, (h2, h1))]\n    # ... 省略中间代码 ...\n\n    # 找最小值的逻辑：\n    min_idx = 0\n    for i in range(1, len(frontier)):\n        # Python 比较 (h2, h1) 时，会自动先比较 h2，如果 h2 相等才会去比较 h1\n        if frontier[i][1] < frontier[min_idx][1]: \n            min_idx = i\n\n    # ... 扩展子节点时加入 Frontier 的逻辑：\n    frontier.append((child, (child_h2, child_h1)))\n\n===================================================================\n"

### **Exercice 6**

Appliquez votre fonction A_etoile pour résoudre un Taquin 3x3 avec l'état initial [[1, 2, 3], [4, 5, 6], [0, 7, 8]] et les deux heuristiques.

Imprimez :
1. Le nombre de nœuds explorés par l'algorithme.
2. Le chemin permettant de passer de l'état initial à l'état objectif (la liste des actions effectuées).
3. Que pouvez-vous observer entre les deux heuristiques ?

In [29]:

initial_state_ex6 = [
    [1, 2, 3],
    [4, 5, 6],
    [0, 7, 8]
]

goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

problem_ex6 = Taquin(initial_state_ex6, goal_state, 3)

print("=== Résolution avec A* et H1 (Tuiles mal placées) ===")
node_h1, count_h1 = A_etoile(problem_ex6, calcule_h1)

if node_h1:
    print(f"Nombre de nœuds explorés : {count_h1}")
    print(f"Solution (Chemin) : {node_h1.solution()}")
    print(f"Coût du chemin : {node_h1.path_cost}")
else:
    print("Pas de solution trouvée.")

print("\n" + "="*50 + "\n")

print("=== Résolution avec A* et H2 (Distance de Manhattan) ===")
node_h2, count_h2 = A_etoile(problem_ex6, calcule_h2)

if node_h2:
    print(f"Nombre de nœuds explorés : {count_h2}")
    print(f"Solution (Chemin) : {node_h2.solution()}")
    print(f"Coût du chemin : {node_h2.path_cost}")
else:
    print("Pas de solution trouvée.")

=== Résolution avec A* et H1 (Tuiles mal placées) ===
Nombre de nœuds explorés : 3
Solution (Chemin) : ['right', 'right']
Coût du chemin : 2


=== Résolution avec A* et H2 (Distance de Manhattan) ===
Nombre de nœuds explorés : 3
Solution (Chemin) : ['right', 'right']
Coût du chemin : 2


# **Partie 3 : Comparaison empirique de DFS, BFS, A etoile avec H1 et A etoile avec H2**

Nous allons comparer les quatre algorithmes sur les 15 instances du jeu de Taquin de l'exercice 2. Pour cela, voici les résultats de BFS et DFS sur ces 15 instances. Attention au format des tableaux. Les noms des colonnes peuvent différer de ceux que vous avez utilisés. De plus, un path_length égal à -1 signifie que l'algorithme n'a pas trouvé la solution avant d'atteindre le budget d'exploration.

In [30]:
results_BFS = [
{'initial_state': [[1, 2, 3], [4, 5, 0], [6, 7, 8]],'algorithm': 'breadth_first_search', 'num_explorations': 1846, 'path_length': 13, 'execution_time': 0.2773609161376953},
{'initial_state': [[1, 2, 3], [0, 5, 6], [4, 7, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 6, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 2, 5], [7, 8, 6]], 'algorithm': 'breadth_first_search', 'num_explorations': 7, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 5, 2], [7, 6, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 10911, 'path_length': 17, 'execution_time': 8.975327491760254},
{'initial_state': [[1, 0, 3], [4, 2, 5], [6, 7, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 4960, 'path_length': 15, 'execution_time': 1.831636905670166},
{'initial_state': [[1, 2, 3], [4, 0, 6], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 3, 'path_length': 2, 'execution_time': 0.0},
{'initial_state': [[1, 2, 3], [0, 4, 6], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 8, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 2, 3], [4, 6, 0], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 8, 'path_length': 3, 'execution_time': 0.0},
{'initial_state': [[1, 3, 6], [4, 2, 5], [7, 0, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 36.05873966217041},
{'initial_state': [[1, 3, 6], [4, 2, 0], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20, 'path_length': 5, 'execution_time': 0.0},
{'initial_state': [[1, 3, 6], [4, 0, 2], [7, 5, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 62, 'path_length': 6, 'execution_time': 0.0019948482513427734},
{'initial_state': [[3, 1, 2], [4, 6, 5], [7, 0, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 40.92391228675842},
{'initial_state': [[8, 1, 2], [0, 4, 3], [7, 6, 5]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 40.67150044441223},
{'initial_state': [[1, 4, 2], [7, 0, 6], [5, 3, 8]], 'algorithm': 'breadth_first_search', 'num_explorations': 3321, 'path_length': 14, 'execution_time': 0.8201050758361816},
{'initial_state': [[2, 8, 3], [1, 6, 4], [7, 0, 5]], 'algorithm': 'breadth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 42.47766828536987}
]
results_DFS = [
{'initial_state': [[1, 2, 3], [4, 5, 0], [6, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 69.51113247871399},
{'initial_state': [[1, 2, 3], [0, 5, 6], [4, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 27, 'path_length': 27, 'execution_time': 0.0},
{'initial_state': [[1, 0, 3], [4, 2, 5], [7, 8, 6]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 60.40420460700989},
{'initial_state': [[1, 0, 3], [4, 5, 2], [7, 6, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 62.47419261932373},
{'initial_state': [[1, 0, 3], [4, 2, 5], [6, 7, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 67.59605073928833},
{'initial_state': [[1, 2, 3], [4, 0, 6], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 412, 'path_length': 406, 'execution_time': 0.02211737632751465},
{'initial_state': [[1, 2, 3], [0, 4, 6], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 65.1717848777771},
{'initial_state': [[1, 2, 3], [4, 6, 0], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 886, 'path_length': 871, 'execution_time': 0.08270859718322754},
{'initial_state': [[1, 3, 6], [4, 2, 5], [7, 0, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 66.5749568939209},
{'initial_state': [[1, 3, 6], [4, 2, 0], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 65.61762142181396},
{'initial_state': [[1, 3, 6], [4, 0, 2], [7, 5, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 66.86844420433044},
{'initial_state': [[3, 1, 2], [4, 6, 5], [7, 0, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 80.54781603813171},
{'initial_state': [[8, 1, 2], [0, 4, 3], [7, 6, 5]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 75.41818499565125},
{'initial_state': [[1, 4, 2], [7, 0, 6], [5, 3, 8]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 69.51109766960144},
{'initial_state': [[2, 8, 3], [1, 6, 4], [7, 0, 5]], 'algorithm': 'depth_first_search', 'num_explorations': 20000, 'path_length': -1, 'execution_time': 83.47795271873474}
]

### **Exercice 7**
Utilisez (et adaptez si nécessaire) votre fonction compare_algorithms du premier TP pour résoudre tous les jeux avec les deux versions de l'algorithme A etoile.

In [31]:
import time

results_misplace = []  #  A* + H1
results_manhattan = [] #  A* + H2 

print("Calcul en cours ...")

for i, state in enumerate(problem_states):
    problem = Taquin(state, goal_state, 3)
    
    # H1
    start_time = time.time()
    node_h1, count_h1 = A_etoile(problem, calcule_h1, budget=20000)
    end_time = time.time()
    
    # result H1
    if node_h1:
        path_len_h1 = len(node_h1.solution())
    else:
        path_len_h1 = -1
        
    results_misplace.append({
        'initial_state': state,
        'algorithm': 'A* H1',
        'num_explorations': count_h1,
        'path_length': path_len_h1,
        'execution_time': end_time - start_time
    })

    # H2
    start_time = time.time()
    node_h2, count_h2 = A_etoile(problem, calcule_h2, budget=20000)
    end_time = time.time()
    
    if node_h2:
        path_len_h2 = len(node_h2.solution())
    else:
        path_len_h2 = -1
        
    results_manhattan.append({
        'initial_state': state,
        'algorithm': 'A* H2',
        'num_explorations': count_h2,
        'path_length': path_len_h2,
        'execution_time': end_time - start_time
    })

print("Terminé !")

Calcul en cours ...
Terminé !


#### **Utilisez le code ci-dessous pour visualiser vos résultats et commentez**

In [32]:
import pandas as pd
import numpy as np
from tabulate import tabulate ## La librairie Tabulate doit être installée. Si vous ne l'avez pas, vous pouvez simplement utiliser "print(df)" pour visualiser le tableau.

Data = []
for i in range(len(problem_states)):
    Data.append([int(i+1),
                 results_BFS[i]['num_explorations'], results_DFS[i]['num_explorations'], results_misplace[i]['num_explorations'], results_manhattan[i]['num_explorations'],
                 results_BFS[i]['path_length'], results_DFS[i]['path_length'], results_misplace[i]['path_length'], results_manhattan[i]['path_length'],
                 round(results_BFS[i]['execution_time'],2), round(results_DFS[i]['execution_time'],2), round(results_misplace[i]['execution_time'],2), round(results_manhattan[i]['execution_time'],2)])

df = pd.DataFrame(Data, columns=["State", "Nodes explored BFS", "Nodes explored DFS", "Nodes explored A*H1", "Nodes explored A*H2",
                                 "Path length BFS", "Path length DFS", "Path length A*H1", "Path length A*H2",
                                 "Execution Time BFS", "Execution Time DFS","Execution Time A*H1", "Execution Time A*H2"])
print(tabulate(df, headers='keys', tablefmt='pretty'))

+----+-------+--------------------+--------------------+---------------------+---------------------+-----------------+-----------------+------------------+------------------+--------------------+--------------------+---------------------+---------------------+
|    | State | Nodes explored BFS | Nodes explored DFS | Nodes explored A*H1 | Nodes explored A*H2 | Path length BFS | Path length DFS | Path length A*H1 | Path length A*H2 | Execution Time BFS | Execution Time DFS | Execution Time A*H1 | Execution Time A*H2 |
+----+-------+--------------------+--------------------+---------------------+---------------------+-----------------+-----------------+------------------+------------------+--------------------+--------------------+---------------------+---------------------+
| 0  |  1.0  |       1846.0       |      20000.0       |        190.0        |        84.0         |      13.0       |      -1.0       |       13.0       |       13.0       |        0.28        |       69.51        | 